In [2]:
import pandas as pd

data = pd.read_csv("Data/dataset_2048_large.csv", dtype = str)

In [5]:
labels = list(data['label'])
labels, len(labels)

(['1000000',
  '1000000',
  '1000000',
  '1111111',
  '0000010',
  '1111111',
  '1100001',
  '0000010',
  '0000010',
  '0000000',
  '1100001',
  '1111111',
  '1111111',
  '1000000',
  '0100111',
  '1000000',
  '1100101',
  '1111111',
  '0000010',
  '0000010',
  '0000010',
  '1111111',
  '0000010',
  '1100001',
  '1100001',
  '1100001',
  '0000010',
  '1000000',
  '1111111',
  '0000010',
  '1111111',
  '0000010',
  '1111111',
  '1100001',
  '1111111',
  '0000010',
  '0000010',
  '1111111',
  '1100101',
  '1111111',
  '1100001',
  '1100101',
  '1100001',
  '1100101',
  '1100101',
  '1000000',
  '0000000',
  '1100101',
  '1101111',
  '0000000',
  '1100001',
  '0000000',
  '1111111',
  '1100001',
  '1111111',
  '1000000',
  '1000000',
  '1100001',
  '1000000',
  '1000000',
  '1111111',
  '1000000',
  '1100101',
  '1111101',
  '0000000',
  '1100001',
  '1111111',
  '1111111',
  '0000010',
  '1100001',
  '1000000',
  '1100101',
  '1100101',
  '1100101',
  '1111111',
  '0000010',
  '1100001',

In [7]:
import numpy as np
# count each occurance of 1s and 0s in position 
count = [0,0,0,0,0,0,0]
for i in labels:
    for j in range(7):
        if i[j] == '1':
            count[j] += 1
np.array(count)/100000


array([0.79032, 0.66123, 0.32991, 0.33908, 0.54032, 0.47309, 0.65276])